<a href="https://colab.research.google.com/github/sc0les/nba_prediction_markets/blob/main/WIP_Data_Acquisition_%26_Cleansing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests as req
import numpy as np
import pandas as pd
import json
import re
import warnings
warnings.filterwarnings('ignore')
from google.colab import drive
drive.mount('/content/drive/')

# Data Collection

The first step in the modeling process was to retrieve the necessary data needed for the project. There are four key data resources that were fetched from various sources.

1. Odds Data: Sports prediction markets are reliant on bookmakers to create betting lines for each game. Incorporation of this data will (1) help us establish target variables, such as Over/Under and Spread hits and (2) enable the creation of implied probablities and adjust modeling inputs to take these external factors into consideration.


2. Team Data: This dataset provides high-level information about each NBA team. It will be needed to properly join the proceeding datasets with the Odds Data.


3. Team Game Data: These stats are aggregate team performance for each game within each season. They will act as a core part of our feature creation.


4. Player Data: These stats, while also on a per-game and per-season basis, are for each individual player who played in each game. The hope is that this more granular datapoints can provide a more granular feature set within model development.

## Odds Data
The data for historical bookmaker odds was not terribly easy to find. From my knowledge, there are no reliable APIs to fetch this information.

Luckily, _________ has a collection of flat files of these information. 

In [ ]:
# odds data was only easily available via flat files
file_list = [
    '/content/drive/My Drive/Sam Colelli BSTN Capstone/Datasets/nba odds 2007-08.xlsx',
    '/content/drive/My Drive/Sam Colelli BSTN Capstone/Datasets/nba odds 2008-09.xlsx',
    '/content/drive/My Drive/Sam Colelli BSTN Capstone/Datasets/nba odds 2009-10.xlsx',
    '/content/drive/My Drive/Sam Colelli BSTN Capstone/Datasets/nba odds 2010-11.xlsx',
    '/content/drive/My Drive/Sam Colelli BSTN Capstone/Datasets/nba odds 2011-12.xlsx',
    '/content/drive/My Drive/Sam Colelli BSTN Capstone/Datasets/nba odds 2012-13.xlsx',
    '/content/drive/My Drive/Sam Colelli BSTN Capstone/Datasets/nba odds 2013-14.xlsx',
    '/content/drive/My Drive/Sam Colelli BSTN Capstone/Datasets/nba odds 2014-15.xlsx',
    '/content/drive/My Drive/Sam Colelli BSTN Capstone/Datasets/nba odds 2015-16.xlsx',
    '/content/drive/My Drive/Sam Colelli BSTN Capstone/Datasets/nba odds 2016-17.xlsx',
    '/content/drive/My Drive/Sam Colelli BSTN Capstone/Datasets/nba odds 2017-18.xlsx',
    '/content/drive/My Drive/Sam Colelli BSTN Capstone/Datasets/nba odds 2018-19.xlsx',
    '/content/drive/My Drive/Sam Colelli BSTN Capstone/Datasets/nba odds 2019-20.xlsx',
    '/content/drive/My Drive/Sam Colelli BSTN Capstone/Datasets/nba odds 2020-21.xlsx',
    '/content/drive/My Drive/Sam Colelli BSTN Capstone/Datasets/nba odds 2021-22.xlsx'
]

lines_list = []

# including a regex to extract date from file name to signify playing season
for file in file_list:
    tmp_df = pd.read_excel(file)
    year = re.findall('\d{4}', file)
    tmp_df['year'] = f'{year}'
    lines_list.append(tmp_df)
    
    
odds_df = pd.concat(lines_list, axis = 0, ignore_index = True)

In [ ]:
odds_df

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H,year
0,1030,501,V,Portland,26,23,28,20,97,184,189.5,900,95,['2007']
1,1030,502,H,SanAntonio,29,30,22,25,106,12.5,13,-1400,5,['2007']
2,1030,503,V,Utah,28,34,24,31,117,214.5,212,100,105.5,['2007']
3,1030,504,H,GoldenState,30,21,21,24,96,3,1,-120,3,['2007']
4,1030,505,V,Houston,16,27,27,25,95,2.5,5,-230,3,['2007']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38311,610,524,H,Boston,28,26,24,19,97,2.5,4,-165,1,['2021']
38312,613,525,V,Boston,16,23,35,20,94,212.5,211,+145,2.5,['2021']
38313,613,526,H,Golden State,27,24,24,29,104,3.5,4,-165,107.5,['2021']
38314,616,527,V,Golden State,27,27,22,27,103,212.5,211.5,+155,103.5,['2021']


In addition to  historical odds data not being readily accessible, the format of the flat files is not great. Additionally, which will be seen further down this notebook, there is no clear field that would easily suffice for a join with the other datasets that will be pulled. Much of the code below is the transforming of data to create temporary IDs to join the datasets.

In [ ]:
# trimming down year to only get the digits
odds_df['year'] = odds_df['year'].str[2:6]

# changing Date to string so I can split it
odds_df['Date'] = odds_df['Date'].astype('str')

#changing year to int so I can add 1 to it to account for season's transition into the new calendar year
odds_df['year'] = odds_df['year'].astype('int')

# grabbing the single digit months to properly split month from day
tmp_short_month = odds_df[odds_df['Date'].str.len() == 3]
tmp_short_month['Month'] = tmp_short_month['Date'].str[0:1]
tmp_short_month['Day'] = tmp_short_month['Date'].str[-2:]

# finals end in June at the latest, and new season starts in October at the earliest, so its safe to add 1 to year for shorter months
tmp_short_month['year'] = tmp_short_month['year'] + 1

tmp_long_month = odds_df[odds_df['Date'].str.len() == 4]
tmp_long_month['Month'] = tmp_long_month['Date'].str[0:2]
tmp_long_month['Day'] = tmp_long_month['Date'].str[-2:]

# joining the temp dataframes
odds_df = pd.concat([tmp_short_month, tmp_long_month], join = 'outer')

odds_df.sort_index()

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H,year,Month,Day
0,1030,501,V,Portland,26,23,28,20,97,184,189.5,900,95,2007,10,30
1,1030,502,H,SanAntonio,29,30,22,25,106,12.5,13,-1400,5,2007,10,30
2,1030,503,V,Utah,28,34,24,31,117,214.5,212,100,105.5,2007,10,30
3,1030,504,H,GoldenState,30,21,21,24,96,3,1,-120,3,2007,10,30
4,1030,505,V,Houston,16,27,27,25,95,2.5,5,-230,3,2007,10,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38311,610,524,H,Boston,28,26,24,19,97,2.5,4,-165,1,2022,6,10
38312,613,525,V,Boston,16,23,35,20,94,212.5,211,+145,2.5,2022,6,13
38313,613,526,H,Golden State,27,24,24,29,104,3.5,4,-165,107.5,2022,6,13
38314,616,527,V,Golden State,27,27,22,27,103,212.5,211.5,+155,103.5,2022,6,16


In [ ]:
#changing year back to a string to convert year, month, day into datetime (cant do it with different dtypes)
odds_df['year'] = odds_df['year'].astype('str')

odds_df['GAME_DATE'] = pd.to_datetime(odds_df['year'] + '/' + odds_df['Month'] + '/' + odds_df['Day'])
odds_df.sort_index()

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H,year,Month,Day,GAME_DATE
0,1030,501,V,Portland,26,23,28,20,97,184,189.5,900,95,2007,10,30,2007-10-30
1,1030,502,H,SanAntonio,29,30,22,25,106,12.5,13,-1400,5,2007,10,30,2007-10-30
2,1030,503,V,Utah,28,34,24,31,117,214.5,212,100,105.5,2007,10,30,2007-10-30
3,1030,504,H,GoldenState,30,21,21,24,96,3,1,-120,3,2007,10,30,2007-10-30
4,1030,505,V,Houston,16,27,27,25,95,2.5,5,-230,3,2007,10,30,2007-10-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38311,610,524,H,Boston,28,26,24,19,97,2.5,4,-165,1,2022,6,10,2022-06-10
38312,613,525,V,Boston,16,23,35,20,94,212.5,211,+145,2.5,2022,6,13,2022-06-13
38313,613,526,H,Golden State,27,24,24,29,104,3.5,4,-165,107.5,2022,6,13,2022-06-13
38314,616,527,V,Golden State,27,27,22,27,103,212.5,211.5,+155,103.5,2022,6,16,2022-06-16


In [ ]:
odds_df.nunique()

Date          326
Rot           368
VH              3
Team           35
1st            51
2nd            50
3rd            50
4th            46
Final         105
Open          213
Close         206
ML            958
2H            126
year           16
Month          12
Day            31
GAME_DATE    3054
dtype: int64

In [ ]:
#odds_df has older team names prior to relocation, along with some different team name formats. cleaning it up to get 30 teams
odds_df = odds_df.replace('Seattle', 'OklahomaCity')
odds_df = odds_df.replace('Oklahoma City', 'OklahomaCity')
odds_df = odds_df.replace('NewJersey', 'Brooklyn')
odds_df = odds_df.replace('Golden State', 'GoldenState')
odds_df = odds_df.replace('LA Clippers', 'LAClippers')

In [ ]:
#creating a separate df of unique team names in odds_df to join with nba_teams_df
odds_team_list = odds_df['Team'].unique()
odds_team_df = pd.DataFrame(odds_team_list)
odds_team_df = odds_team_df.sort_values(0)
odds_team_df.reset_index(drop = True, inplace = True)

## Team Data
Unlike the historical bookmaker dataset, NBA historical stats are incredibly easy to fetch via the NBA's APIs and the python packages that are specifically design with these external services.

In [ ]:
from nba_api.stats.static import teams
nba_teams = teams.get_teams()
nba_teams_df = pd.DataFrame(nba_teams)
nba_teams_df = nba_teams_df.sort_values('full_name')
nba_teams_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 0 to 27
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            30 non-null     int64 
 1   full_name     30 non-null     object
 2   abbreviation  30 non-null     object
 3   nickname      30 non-null     object
 4   city          30 non-null     object
 5   state         30 non-null     object
 6   year_founded  30 non-null     int64 
dtypes: int64(2), object(5)
memory usage: 1.9+ KB


In [ ]:
nba_teams_df.sort_values('full_name')
nba_teams_df.reset_index(drop = True, inplace = True)

In [ ]:
# now that both datasets have the same 30 teams, we can map the naming conventions to a unique identifing team ID, id.
nba_teams_df['Team'] = odds_team_df[0]
nba_teams_df

,id,full_name,abbreviation,nickname,city,state,year_founded,Team
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Atlanta,1949,Atlanta
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946,Boston
2,1610612751,Brooklyn Nets,BKN,Nets,Brooklyn,New York,1976,Brooklyn
3,1610612766,Charlotte Hornets,CHA,Hornets,Charlotte,North Carolina,1988,Charlotte
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966,Chicago
5,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970,Cleveland
6,1610612742,Dallas Mavericks,DAL,Mavericks,Dallas,Texas,1980,Dallas
7,1610612743,Denver Nuggets,DEN,Nuggets,Denver,Colorado,1976,Denver
8,1610612765,Detroit Pistons,DET,Pistons,Detroit,Michigan,1948,Detroit
9,1610612744,Golden State Warriors,GSW,Warriors,Golden State,California,1946,GoldenState


In [ ]:
odds_df = pd.merge(odds_df, nba_teams_df, on = 'Team', how = 'inner')
odds_df

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,...,Month,Day,GAME_DATE,id,full_name,abbreviation,nickname,city,state,year_founded
0,102,701,V,Brooklyn,18,20,28,30,96,199.5,...,1,02,2008-01-02,1610612751,Brooklyn Nets,BKN,Nets,Brooklyn,New York,1976
1,104,712,H,Brooklyn,21,21,27,33,102,7,...,1,04,2008-01-04,1610612751,Brooklyn Nets,BKN,Nets,Brooklyn,New York,1976
2,105,501,V,Brooklyn,22,28,26,37,113,187,...,1,05,2008-01-05,1610612751,Brooklyn Nets,BKN,Nets,Brooklyn,New York,1976
3,108,701,V,Brooklyn,23,22,29,25,99,1.5,...,1,08,2008-01-08,1610612751,Brooklyn Nets,BKN,Nets,Brooklyn,New York,1976
4,109,508,H,Brooklyn,30,29,21,19,99,7.5,...,1,09,2008-01-09,1610612751,Brooklyn Nets,BKN,Nets,Brooklyn,New York,1976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38311,1223,524,H,LALakers,24,31,37,18,110,1,...,12,23,2021-12-23,1610612747,Los Angeles Lakers,LAL,Lakers,Los Angeles,California,1948
38312,1225,592,H,LALakers,23,39,20,33,115,1.5,...,12,25,2021-12-25,1610612747,Los Angeles Lakers,LAL,Lakers,Los Angeles,California,1948
38313,1228,569,V,LALakers,34,33,30,35,132,4.5,...,12,28,2021-12-28,1610612747,Los Angeles Lakers,LAL,Lakers,Los Angeles,California,1948
38314,1229,507,V,LALakers,30,24,29,16,99,225.5,...,12,29,2021-12-29,1610612747,Los Angeles Lakers,LAL,Lakers,Los Angeles,California,1948


## Team Game Data

In [ ]:
from nba_api.stats.endpoints import leaguegamefinder
from ratelimiter import RateLimiter

games_df = pd.DataFrame(columns = ['SEASON_ID'])

rate_limiter = RateLimiter(max_calls=1, period=7)

for team_id in odds_df['id'].unique():
    with rate_limiter:
        gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable = team_id)
        games = gamefinder.get_data_frames()[0]
        games_df = pd.concat([games_df, games], join = 'outer')

In [ ]:
games_df.reset_index(drop = True, inplace = True)

In [ ]:
games_df

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22022,1.610613e+09,BKN,Brooklyn Nets,0022200221,2022-11-17,BKN @ POR,W,239.0,109.0,...,0.783,5.0,42.0,47.0,32.0,4.0,5.0,17.0,24.0,2.0
1,22022,1.610613e+09,BKN,Brooklyn Nets,0022200210,2022-11-15,BKN @ SAC,L,240.0,121.0,...,0.909,3.0,26.0,29.0,25.0,5.0,3.0,17.0,18.0,-32.0
2,22022,1.610613e+09,BKN,Brooklyn Nets,0022200198,2022-11-13,BKN @ LAL,L,241.0,103.0,...,0.774,8.0,33.0,41.0,22.0,5.0,5.0,12.0,17.0,-13.0
3,22022,1.610613e+09,BKN,Brooklyn Nets,0022200184,2022-11-12,BKN @ LAC,W,242.0,110.0,...,0.667,6.0,35.0,41.0,27.0,9.0,6.0,12.0,18.0,15.0
4,22022,1.610613e+09,BKN,Brooklyn Nets,0022200164,2022-11-09,BKN vs. NYK,W,242.0,112.0,...,0.700,11.0,44.0,55.0,30.0,11.0,5.0,14.0,22.0,27.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100690,21983,1.610613e+09,LAL,Los Angeles Lakers,0028300066,1983-11-08,LAL @ DEN,W,240.0,133.0,...,0.917,22.0,31.0,53.0,30.0,11.0,8.0,23.0,31.0,NaN
100691,21983,1.610613e+09,LAL,Los Angeles Lakers,0028300053,1983-11-05,LAL @ DAL,L,240.0,102.0,...,0.667,18.0,23.0,41.0,29.0,7.0,8.0,20.0,27.0,NaN
100692,21983,1.610613e+09,LAL,Los Angeles Lakers,0028300035,1983-11-02,LAL @ SDC,L,240.0,106.0,...,0.760,16.0,33.0,49.0,29.0,14.0,7.0,24.0,24.0,NaN
100693,21983,1.610613e+09,LAL,Los Angeles Lakers,0028300010,1983-10-29,LAL @ UTH,W,240.0,120.0,...,0.718,29.0,31.0,60.0,25.0,10.0,10.0,20.0,41.0,NaN


In [ ]:
# creating another temporary ID to join the odds and team data with the game-level data.
games_df['TEAM_ID'] = games_df['TEAM_ID'].astype('int')
pd.to_datetime(games_df['GAME_DATE'])
games_df['tmp_id'] = games_df['TEAM_ID'].astype('str') + games_df['GAME_DATE']

In [ ]:
odds_df['tmp_id'] = odds_df['id'].astype('str') + odds_df['GAME_DATE'].astype('str')

In [ ]:
m = pd.merge(odds_df, games_df, on = 'tmp_id', how = 'inner')

In [ ]:
# checking to ensure the merge was successful. each game should only have 2 teams.
team_counts = m.groupby(['GAME_ID'])['GAME_ID'].count()
m[m['GAME_ID'].isin(team_counts[team_counts > 2].index)]

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
21573,1228,549,V,Portland,22,36,26,31,115,227.5,...,0.867,7.0,38.0,45.0,27.0,9.0,6.0,9.0,16.0,8.0
38243,1228,540,H,LALakers,40,27,36,24,127,11,...,0.833,6.0,35.0,41.0,20.0,6.0,5.0,14.0,14.0,-8.0
38244,1228,550,H,LALakers,30,24,31,22,107,4.5,...,0.833,6.0,35.0,41.0,20.0,6.0,5.0,14.0,14.0,-8.0


In [ ]:
m[m['GAME_ID'].isin(team_counts[team_counts < 2].index)]

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
6404,129,515,V,Charlotte,23,19,18,28,88,196,...,0.75,14.0,35.0,49.0,20.0,2.0,5.0,17.0,22.0,-15.0
17887,1009,709,N,Miami,25,35,28,23,111,217,...,1.00,6.0,37.0,43.0,23.0,11.0,4.0,19.0,23.0,14.0


In [ ]:
# relative to the size of the data, these data points will simply be dropped from the dataset.
m.drop([6404, 17887, 38243], inplace = True)

In [ ]:
m.sort_values('GAME_ID')

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
21170,1030,501,V,Portland,26,23,28,20,97,184,...,0.765,8.0,32.0,40.0,15.0,1.0,4.0,16.0,19.0,-9.0
31408,1030,502,H,SanAntonio,29,30,22,25,106,12.5,...,0.692,12.0,28.0,40.0,21.0,8.0,4.0,8.0,19.0,9.0
13458,1030,505,V,Houston,16,27,27,25,95,2.5,...,0.677,12.0,37.0,49.0,23.0,10.0,5.0,18.0,30.0,2.0
37846,1030,506,H,LALakers,25,18,19,31,93,191,...,0.600,11.0,26.0,37.0,18.0,16.0,3.0,12.0,22.0,-2.0
23700,1030,504,H,GoldenState,30,21,21,24,96,3,...,0.684,7.0,30.0,37.0,19.0,8.0,9.0,20.0,28.0,-21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31407,413,507,V,SanAntonio,22,28,25,28,103,230.5,...,0.727,8.0,26.0,34.0,19.0,4.0,3.0,6.0,20.0,-10.0
9613,415,510,H,Cleveland,36,25,23,17,101,226,...,0.824,8.0,28.0,36.0,23.0,8.0,4.0,11.0,20.0,-6.0
8302,415,509,V,Atlanta,25,26,33,23,107,1.5,...,0.909,10.0,32.0,42.0,15.0,5.0,3.0,13.0,19.0,6.0
30066,415,512,H,LAClippers,22,24,38,17,101,5,...,0.647,12.0,33.0,45.0,20.0,7.0,3.0,7.0,21.0,-4.0


#### WARNING: The below API requests are both rate limited and a very large amount of data. It took ~24 hours to fully fetch all player game stats.

In [ ]:
from nba_api.stats.endpoints import boxscoreadvancedv2
from time import sleep

game_list = m['GAME_ID'].unique().tolist()
game_stats_df = pd.DataFrame(columns = ['GAME_ID'])

rate_limiter = RateLimiter(max_calls=1, period=8)

for game in game_list[19123:]:
    with rate_limiter:
        boxscore = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id = game).player_stats.get_data_frame()
        game_stats_df = pd.concat([game_stats_df, boxscore], join = 'outer')

In [ ]:
game_stats_df = pd.read_csv('/content/drive/My Drive/Sam Colelli BSTN Capstone/Datasets/game_stats.csv', index_col = 0)
game_stats_df

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,...,TM_TOV_PCT,EFG_PCT,TS_PCT,USG_PCT,E_USG_PCT,E_PACE,PACE,PACE_PER40,POSS,PIE
0,20700459,1.610613e+09,NJN,New Jersey,2210.0,Richard Jefferson,Richard,F,NaN,38:37,...,14.3,0.333,0.416,0.224,0.223,95.49,92.60,77.17,74.0,0.034
1,20700459,1.610613e+09,NJN,New Jersey,201157.0,Sean Williams,Sean,F,NaN,16:00,...,0.0,0.250,0.225,0.125,0.123,90.54,90.00,75.00,29.0,-0.010
2,20700459,1.610613e+09,NJN,New Jersey,200767.0,Josh Boone,Josh,C,NaN,16:07,...,0.0,0.250,0.410,0.143,0.144,88.22,87.86,73.22,29.0,0.010
3,20700459,1.610613e+09,NJN,New Jersey,1713.0,Vince Carter,Vince,G,NaN,41:36,...,3.7,0.382,0.458,0.236,0.235,94.36,90.58,75.48,78.0,0.116
4,20700459,1.610613e+09,NJN,New Jersey,467.0,Jason Kidd,Jason,G,NaN,37:05,...,14.3,0.375,0.512,0.159,0.158,95.06,91.90,76.58,71.0,0.155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,21900139,1.610613e+09,LAL,Los Angeles,1627936.0,Alex Caruso,Alex,NaN,NaN,17:22,...,11.1,0.333,0.387,0.119,0.127,102.60,102.26,85.22,38.0,0.045
21,21900139,1.610613e+09,LAL,Los Angeles,203584.0,Troy Daniels,Troy,NaN,NaN,17:19,...,0.0,0.786,0.786,0.175,0.184,100.18,102.56,85.47,38.0,0.139
22,21900139,1.610613e+09,LAL,Los Angeles,201162.0,Jared Dudley,Jared,NaN,NaN,2:02,...,0.0,0.000,0.000,0.000,0.000,93.01,106.23,88.52,4.0,0.000
23,21900139,1.610613e+09,LAL,Los Angeles,2730.0,Dwight Howard,Dwight,NaN,NaN,16:51,...,0.0,1.000,0.865,0.122,0.129,101.81,98.28,81.90,35.0,0.126


# Data Cleansing

Now that the 4 datasets are joined, the next step is cleansing the data in preparation for modeling. The snippets below are mainly centered around the changing of datatypes, dropping of columns, etc.

In [ ]:
m.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38280 entries, 0 to 38282
Data columns (total 53 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Date               38280 non-null  object        
 1   Rot                38280 non-null  int64         
 2   VH                 38280 non-null  object        
 3   Team               38280 non-null  object        
 4   1st                38280 non-null  int64         
 5   2nd                38280 non-null  int64         
 6   3rd                38280 non-null  int64         
 7   4th                38280 non-null  int64         
 8   Final              38280 non-null  int64         
 9   Open               38280 non-null  object        
 10  Close              38277 non-null  object        
 11  ML                 38280 non-null  object        
 12  2H                 38278 non-null  object        
 13  year               38280 non-null  object        
 14  Month 

In [ ]:
# quick drop of some repeated and unnecessary columns.
m.drop(columns = ['Date', 'Team', 'nickname', 'year_founded', 'tmp_id'], inplace = True)

In [ ]:
# when attempting to convert the betting lines to numeric, I ran into a number of values that needed to be adjusted
m['Open'].replace('197.5u10', 197.5, inplace = True)
m['Open'].replace('pk', 0, inplace = True)
m['Open'].replace('PK', 0, inplace = True)
m['Close'].replace('pk', 0, inplace = True)
m['Close'].replace('PK', 0, inplace = True)
m['ML'].replace('NL', 0, inplace = True)
m['ML'].replace('+145\xa0', 145, inplace = True)
m['ML'].replace('+155\xa0', 155, inplace = True)
m['ML'].replace('+145', 145, inplace = True)
m['ML'].replace('+155', 155, inplace = True)
m['ML'].replace('+145 ', 145, inplace = True)
m['ML'].replace('+155 ', 155, inplace = True)
m['2H'].replace('pk', 100, inplace = True)
m['2H'].replace('PK', 100, inplace = True)
m['2H'].replace('NL', 0, inplace = True)
m['2H'].replace('.5ev', 0, inplace = True)
m['2H'].replace('.5+03', 0, inplace = True)

# this scripts were used to identify which values were causing the datatype conversion issues:
import logging as log

for col in m[['Open', 'Close', 'ML', '2H']]:
    try:
        m[col] = pd.to_numeric(m[col])
    except:
        mask = pd.to_numeric(m[col].fillna('0'), errors = 'coerce').isna()
        l = m.loc[mask, col].tolist()
        for val in l:
            print(f'Not converted values in {col}: {val}')
            log.exception(f'Not converted values in {col}: {val}')

In the Odds dataset, there was no differentiation between the Spread and the Over/Under stat lines. Both were stored under `Open` and `Close`. Luckily, these numbers are wildly different in terms of lower and upper bounds on the number line (Spreads are always less than overall game totals).

The code blocks below are leveraging groupbys and column creations and renamings to clearly signify the difference between the two betting lines.

In [ ]:
spreads = m.groupby('GAME_ID')[['Open', 'Close', '2H']].min()
spreads.rename(columns = {'Open': 'spread_open', 'Close': 'spread_close', '2H': 'spread_2H'}, inplace = True)

totals = m.groupby('GAME_ID')[['Open', 'Close', '2H']].max()
totals.rename(columns = {'Open': 'total_open', 'Close': 'total_close', '2H': 'total_2H'}, inplace = True)

In [ ]:
m.drop(columns = ['Open', 'Close', '2H'], inplace = True)

The last aspect centrally focused on data cleansing is the clear identification of each team playing in a given game. The `VH` column, while mostly sufficing, has 3 values to signify Home Team (`H`), Visiting Team (`V`), and Neutral (`N`). Neutral represents games that are not played at either team's court, and instead are playing at a neutral location. 

Since the final dataset schema we wish to move forward with is highly dependent on `H` and `V` status, we need to find another way to replace `N` with an appropriate value.

We'll leverage the `home_away` field, which has each game matchup with their appropriate `<Home Team> vs. <Away Team>` and `<Away Team> @ <Home Team>`, regardless of if the game was played at a team-agnostic venue.

In [ ]:
home_team_df = m[m['VH'] == 'H']
home_team_df

,Rot,VH,1st,2nd,3rd,4th,Final,ML,year,Month,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
1,712,H,21,21,27,33,102,-245.0,2008,1,...,0.679,10.0,33.0,43.0,31.0,13.0,5.0,12.0,21.0,6.0
4,508,H,30,29,21,19,99,-360.0,2008,1,...,0.682,10.0,32.0,42.0,27.0,13.0,4.0,15.0,18.0,11.0
5,810,H,30,14,24,9,77,250.0,2008,1,...,0.375,19.0,33.0,52.0,22.0,4.0,6.0,15.0,23.0,-9.0
6,706,H,12,22,19,20,73,-205.0,2008,1,...,0.773,15.0,23.0,38.0,18.0,6.0,3.0,9.0,15.0,-26.0
7,712,H,33,22,23,27,105,-335.0,2008,1,...,0.645,19.0,24.0,43.0,28.0,5.0,2.0,16.0,30.0,-6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38273,576,H,18,31,36,21,106,-430.0,2021,12,...,0.647,6.0,38.0,44.0,24.0,12.0,11.0,13.0,17.0,12.0
38277,558,H,25,27,21,17,90,250.0,2021,12,...,0.652,6.0,41.0,47.0,14.0,9.0,3.0,14.0,18.0,-18.0
38278,524,H,24,31,37,18,110,-125.0,2021,12,...,0.567,14.0,34.0,48.0,21.0,4.0,6.0,14.0,20.0,-28.0
38279,592,H,23,39,20,33,115,-135.0,2021,12,...,0.893,12.0,35.0,47.0,29.0,8.0,5.0,13.0,23.0,-7.0


In [ ]:
away_team_df = m[m['VH'] == 'V']
away_team_df

,Rot,VH,1st,2nd,3rd,4th,Final,ML,year,Month,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,701,V,18,20,28,30,96,220.0,2008,1,...,0.714,9.0,33.0,42.0,24.0,5.0,4.0,13.0,24.0,1.0
2,501,V,22,28,26,37,113,160.0,2008,1,...,0.652,10.0,31.0,41.0,28.0,11.0,3.0,17.0,22.0,6.0
3,701,V,23,22,29,25,99,-125.0,2008,1,...,0.632,7.0,29.0,36.0,22.0,7.0,3.0,10.0,25.0,-16.0
8,521,V,19,26,27,29,107,-110.0,2008,1,...,0.765,20.0,34.0,54.0,26.0,6.0,8.0,17.0,26.0,-13.0
9,801,V,20,25,28,19,92,500.0,2008,1,...,0.815,5.0,33.0,38.0,20.0,6.0,6.0,16.0,15.0,-24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38274,533,V,33,14,26,20,107,-160.0,2021,12,...,0.778,4.0,42.0,46.0,24.0,8.0,2.0,14.0,20.0,3.0
38275,565,V,16,29,20,27,92,-120.0,2021,12,...,0.690,1.0,35.0,36.0,26.0,6.0,7.0,18.0,22.0,-18.0
38276,515,V,22,37,25,26,110,240.0,2021,12,...,0.636,9.0,37.0,46.0,25.0,6.0,8.0,19.0,25.0,-5.0
38280,569,V,34,33,30,35,132,-220.0,2021,12,...,0.759,14.0,34.0,48.0,27.0,8.0,6.0,12.0,23.0,9.0


In [ ]:
m[m['VH'] == 'N'].sort_values('GAME_ID')

,Rot,VH,1st,2nd,3rd,4th,Final,ML,year,Month,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
179,802,N,27,20,31,38,116,-145.0,2011,3,...,0.760,7.0,34.0,41.0,24.0,3.0,7.0,11.0,23.0,13.0
35870,801,N,23,28,30,22,103,125.0,2011,3,...,0.792,7.0,29.0,36.0,26.0,5.0,2.0,6.0,22.0,-13.0
180,502,N,23,42,23,22,137,-130.0,2011,3,...,0.744,20.0,42.0,62.0,34.0,6.0,14.0,16.0,30.0,1.0
35871,501,N,30,32,26,22,136,110.0,2011,3,...,0.889,16.0,38.0,54.0,27.0,7.0,5.0,7.0,32.0,-1.0
2797,502,N,17,24,22,24,87,190.0,2013,1,...,0.500,11.0,25.0,36.0,22.0,7.0,4.0,15.0,21.0,-15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37725,712,N,33,28,26,30,117,-200.0,2020,9,...,0.828,5.0,38.0,43.0,24.0,6.0,8.0,13.0,26.0,10.0
17361,701,N,28,20,19,31,98,165.0,2020,9,...,0.786,5.0,31.0,36.0,23.0,4.0,5.0,8.0,15.0,-18.0
37726,702,N,31,34,28,23,116,-185.0,2020,9,...,0.926,9.0,45.0,54.0,26.0,7.0,8.0,12.0,19.0,18.0
21043,724,N,31,27,31,37,126,-260.0,2020,8,...,0.854,9.0,30.0,39.0,20.0,12.0,7.0,13.0,19.0,4.0


In [ ]:
m[['team', 'home_away', 'opp']] = m['MATCHUP'].str.split(' ', expand = True)
m['home_away'].replace('vs.', 'H', inplace = True)
m['home_away'].replace('@', 'V', inplace = True)
m

,Rot,VH,1st,2nd,3rd,4th,Final,ML,year,Month,...,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,team,home_away,opp
0,701,V,18,20,28,30,96,220.0,2008,1,...,42.0,24.0,5.0,4.0,13.0,24.0,1.0,NJN,V,ORL
1,712,H,21,21,27,33,102,-245.0,2008,1,...,43.0,31.0,13.0,5.0,12.0,21.0,6.0,NJN,H,CHA
2,501,V,22,28,26,37,113,160.0,2008,1,...,41.0,28.0,11.0,3.0,17.0,22.0,6.0,NJN,V,ATL
3,701,V,23,22,29,25,99,-125.0,2008,1,...,36.0,22.0,7.0,3.0,10.0,25.0,-16.0,NJN,V,CHA
4,508,H,30,29,21,19,99,-360.0,2008,1,...,42.0,27.0,13.0,4.0,15.0,18.0,11.0,NJN,H,SEA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38278,524,H,24,31,37,18,110,-125.0,2021,12,...,48.0,21.0,4.0,6.0,14.0,20.0,-28.0,LAL,H,SAS
38279,592,H,23,39,20,33,115,-135.0,2021,12,...,47.0,29.0,8.0,5.0,13.0,23.0,-7.0,LAL,H,BKN
38280,569,V,34,33,30,35,132,-220.0,2021,12,...,48.0,27.0,8.0,6.0,12.0,23.0,9.0,LAL,V,HOU
38281,507,V,30,24,29,16,99,210.0,2021,12,...,39.0,24.0,5.0,5.0,18.0,25.0,-5.0,LAL,V,MEM


In [ ]:
m = pd.merge(m, spreads, on = 'GAME_ID', how = 'outer')
m = pd.merge(m, totals, on = 'GAME_ID', how = 'outer')

In [ ]:
#drop of repeated or unnecessary columns due to transformations above
m.drop(columns = ['Rot', 'VH', 'team', 'GAME_DATE_x', 'abbreviation', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'MATCHUP', 'full_name', 'city', 'state', 'opp'], inplace = True)

In [ ]:
m.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38280 entries, 0 to 38279
Data columns (total 42 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   1st           38280 non-null  int64  
 1   2nd           38280 non-null  int64  
 2   3rd           38280 non-null  int64  
 3   4th           38280 non-null  int64  
 4   Final         38280 non-null  int64  
 5   ML            38280 non-null  float64
 6   year          38280 non-null  object 
 7   Month         38280 non-null  object 
 8   Day           38280 non-null  object 
 9   id            38280 non-null  int64  
 10  SEASON_ID     38280 non-null  object 
 11  TEAM_ID       38280 non-null  int64  
 12  GAME_ID       38280 non-null  object 
 13  GAME_DATE_y   38280 non-null  object 
 14  WL            38280 non-null  object 
 15  MIN           38280 non-null  float64
 16  PTS           38280 non-null  float64
 17  FGM           38280 non-null  float64
 18  FGA           38280 non-nu

In [ ]:
m.to_csv('full_game_stats.csv')